# Контрольная работа №1, второй семестр

## Задача B

Для решения задачи необходимо было придумать линейный алгоритм нахождения циклического сдвига. Для этого нужно посчитать Z-функцию для строки `A + "#" + B + B`. Ответом была первая позиция в массиве (не забываем сначала убрать ненужый кусок с массива со значениями для префикса `A + "#"`), где значение Z-функции равно `|A|`. Аналогично можно было применить префикс функцию.

## Задача D

Решение задачи требует использование алгоритма Левештейна, который позволяет вычислить редакционное расстояние. Алгоритм представляет из себя несложную двумерную динамику.

In [1]:
if __name__ == "__main__":
    s = input()
    t = input()
    n = len(s) + 1
    m = len(t) + 1
    d = [[0] * m for _ in range(n)]
    for i in range(n):
        d[i][0] = i
    for j in range(m):
        d[0][j] = j
    for i in range(1, n):
        for j in range(1, m):
            d[i][j] = min(d[i][j - 1] + 1,
                          d[i - 1][j] + 1,
                          d[i - 1][j - 1] + (1 if s[i - 1] != t[j - 1] else 0))
    print(d[n - 1][m - 1])

abcdef
cdefab
4


## Задача F

Данная задача проста в написании, но требует хорошего понимания условия. В задаче необходимо найти матожидание количества потомков, имеющих хотя бы одну `А` в генотипе. Опустим принципы наследования, это школьная тема. Посмотрим на количество возможных потомков, имеющих хотя бы одну `A` для каждой возможной пары родителей: 4, 4, 4, 3, 2 и 0 соответственно. Считая, что потомок каждого типа может появиться с равной вероятностью, посмотрим на вероятность появления искомого потомка в каждой группе: 1, 1, 1, 0.75, 0.5 и 0 соответсвтенно. Дальше просто необходимо умножить вероятность на удвоенное (т.к. по условию задачи 2 потомка у каждой пары) количество пар в каждой категории и просуммировать.

In [2]:
if __name__ == "__main__":
    a = map(int, input().strip().split())
    c = 0
    for i, j in zip(a, [2, 2, 2, 1.5, 1, 0]):
        c += i * j
    print(c)

1 0 0 1 0 1
3.5


## Задача G

Для решение задачи необходимо было найти подстроку длины равной сумме длин шаблонов. Подстрока должна состоять из шаблонов, как из подпоследовательностей. Т.е. относительный порядок символов внутри каждой подпоследовательности **не меняется**. Просто перебираем подстроки, пытаясь рекурсивно расплести ее на шаблоны. Возможная оптимизация: проверять множества символов в подстроке и шаблонах. Если не равны, то пропускаем подстроку. 

In [2]:
def check(s, t, u):
    if t == u == "":
        return True
    result = False
    if t and s[0] == t[0]:
        result |= check(s[1:], t[1:], u)
    if not result and u and s[0] == u[0]:
        result |= check(s[1:], t, u[1:])
    return result


if __name__ == "__main__":
    s = input()
    t = input()
    u = input()
    n = len(t)
    m = len(u)
    for i in range(len(s) - n - m):
        if set(s[i:i + n + m]) == set(t) | set(u) and check(s[i:i + n + m], t, u):
            print("Yes")
            exit(0)
    print("No")

GACCACGGTT
ACAG
CCG
Yes


## Задача H

Для решения задачи можно было использовать как питоновские хеши, так и полиномиальные. Однако, следует учесть, что если не ограничивать полиномиальные хеши по значению, решение получит Memory Limit. Тогда просто берем достаточно большое число, которым ограничим значения хешей. Например, 2<sup>64</sup>. И дальше используем остаток от деления на это число. 

## Задача I

Для решения задачи необходимо было написать алгоритм, работающий за линейное время. Работает он в 2 этапа. На первом этапе просто вычислим префикс-функцию от нашей строки. На втором будем вычислять значения B-функции, используя значения префикс-функции с первого этапа. Будем двигаться по массиву $\pi$, поддерживая в переменной $m$ наибольшее пройденное значение префикс-функции. Пусть наш итератор $l = \pi_j$. Рассмотрим 2 случая: $l \le m$ и $l \gt m$.

$l \le m$:
В таком случае мы уже раньше встречали подстроку, совпадающую с $s_{j-\pi_j+1}...s_j$ минимум 2 раза, причем для позиции $j$ длина $\pi_j$ максимальна. Значит значение B-функции для позиции $j$ равно значению префикс-фукнции в этой позиции.
<img src="files/l_le_m.png">

$l \gt m$:
В этом случае подстрока $s_{j-\pi_j+1}...s_j$ встречалась ровно один раз и она является префиксом $s_0...s_{\pi_j-1}$ строки. Тогда просто воспользуемся уже посчитанным значением для позиции $\pi_j-1$.
<img src="files/l_gt_m.png">

In [4]:
def prefix_func(s):
    n = len(s)
    pi = [0] * n
    for i in range(1, n):
        j = pi[i - 1]
        while j > 0 and s[j] != s[i]:
            j = pi[j - 1]
        if s[j] == s[i]:
            j += 1
        pi[i] = j
    return pi


def b_func(s):
    pi = prefix_func(s)
    b = []
    m = 0
    for l in pi:
        if m >= l:
            b.append(l)
        else:
            b.append(pi[l - 1])
        m = max(l, m)
    return b


if __name__ == "__main__":
    s = input()
    print(" ".join(map(str, b_func(s))))

abacabacabacaba
0 0 0 0 1 0 1 0 1 2 3 4 5 6 7


## Задача J

Для решения задачи нужно было придумать способ подсчета подпалиндромов быстрее, чем $O(n^2)$. У задачи есть ряд разных решений. Одним из них является алгоритм Манакера. ПРинцип его работы и доказательство линейности вы можете прочитать в разделе [Нахождение всех подпалиндромов](https://e-maxx.ru/algo/palindromes_count) на e-maxx. Сам алгоритм очень сильно похож на z-функцию, в следствии чего самостоятельный разбор алгоритма не должен даставить трудностей. Стоит заметить, что алгоритм дополнительно имеет возможность поиска палиндромов четной длины, но в задаче нужны только нечетные.

In [5]:
def manaker(s):
    left = right = 0
    n = len(s)
    p = [0] * n
    for i in range(1, n):
        if i <= right:
            p[i] = min(right - i, p[left + right - i])
        while p[i] < i and i + p[i] + 1 < n and\
                s[i - p[i] - 1] == s[i + p[i] + 1]:
            p[i] += 1
        if i + p[i] > right:
            left = i - p[i]
            right = i + p[i]
    for i in range(n):
        p[i] = p[i] * 2 + 1
    return p


if __name__ == "__main__":
    s = input()
    print(" ".join(map(str, manaker(s))))

aaaaa
1 3 5 3 1
